In [1]:
import pandas as pd

# 작업디렉토리 설정

In [2]:
# %cd C:\mimic-iv-2.2\mimic-iv-2.2\hosp

C:\mimic-iv-2.2\mimic-iv-2.2\hosp


In [32]:
%cd /data/dhk/physionet.org/files/mimiciv/2.2/hosp

# d_icd_diagnoses파일

In [3]:
df_d_icd_diagnoses = pd.read_csv('d_icd_diagnoses.csv')
df_d_icd_diagnoses.head()

,icd_code,icd_version,long_title
0,0010,9,Cholera due to vibrio cholerae
1,0011,9,Cholera due to vibrio cholerae el tor
2,0019,9,"Cholera, unspecified"
3,0020,9,Typhoid fever
4,0021,9,Paratyphoid fever A


# diagnoses_icd파일

In [4]:
df_diagnoses_icd = pd.read_csv("diagnoses_icd.csv")
df_diagnoses_icd.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
0,10000032,22595853,1,5723,9
1,10000032,22595853,2,78959,9
2,10000032,22595853,3,5715,9
3,10000032,22595853,4,07070,9
4,10000032,22595853,5,496,9


# 고혈압&암환자

In [5]:
# 고혈압 진단 코드 정규식
high_bp_regex = r'^I1[0-5]$|^401'

#암 진단 코드 정규식  
#ICD-9 코드 분류 (140-239)
#ICD-10 코드 분류 (C00-C96, D00-D09)
cancer_regex = r'^(D(0[0-9])|C([0-8][0-9]|9[0-6])|(1[4-9][0-9]|2[0-3][0-9])).*'


# 암 진단 코드 추출
cancer_df = df_diagnoses_icd[df_diagnoses_icd['icd_code'].str.match(cancer_regex)]

# 암 진단을 받은 환자 ID 추출
cancer_patient_ids = cancer_df['subject_id'].unique()

# 암 진단을 받은 환자들의 데이터 추출
cancer_diagnoses_merged = pd.merge(df_diagnoses_icd, df_d_icd_diagnoses, on=['icd_code'])
cancer_patients_data = cancer_diagnoses_merged[cancer_diagnoses_merged['subject_id'].isin(cancer_patient_ids)]



# 암 및 고혈압을 동시에 가진 환자들의 데이터 추출 (첫 번째 진단만 포함)
cancer_high_bp_ids = cancer_patients_data[cancer_patients_data['icd_code'].str.match(high_bp_regex)]['subject_id'].unique()
cancer_high_bp_df = cancer_diagnoses_merged[(cancer_diagnoses_merged['subject_id'].isin(cancer_high_bp_ids)) & 
                                            (cancer_diagnoses_merged['seq_num'] == 1) & 
                                            (cancer_diagnoses_merged['icd_code'].str.match(cancer_regex))]

cancer_high_bp_df

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title
393337,10071129,25200387,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
393339,10094971,20200109,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
393351,10171405,23985269,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
393356,10171405,29677692,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
393364,10322775,24742385,1,1890,9,9,"Malignant neoplasm of kidney, except pelvis"
...,...,...,...,...,...,...,...
4859732,19128927,28814179,1,C44692,10,10,Other specified malignant neoplasm of skin of ...
4859827,19282415,28695316,1,C004,10,10,"Malignant neoplasm of lower lip, inner aspect"
4859828,19290501,28187167,1,C8468,10,10,"Anaplastic large cell lymphoma, ALK-positive, ..."
4860174,19851929,29152819,1,1643,9,9,Malignant neoplasm of posterior mediastinum


# 첫입원기록만 가져옴

In [26]:
# subject_id로 그룹화하고 각 그룹에서 hadm_id가 가장 작은 행 선택
result = cancer_high_bp_df.loc[cancer_high_bp_df.groupby('subject_id')['hadm_id'].idxmin()]

# 필요시 인덱스 리셋
result.reset_index(drop=True, inplace=True)
result

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title
0,10001401,21544441,1,C675,10,10,Malignant neoplasm of bladder neck
1,10001877,25679292,1,2252,9,9,Benign neoplasm of cerebral meninges
2,10002155,20345487,1,1628,9,9,Malignant neoplasm of other parts of bronchus ...
3,10002348,22725460,1,C7931,10,10,Secondary malignant neoplasm of brain
4,10003400,20214994,1,1543,9,9,"Malignant neoplasm of anus, unspecified site"
...,...,...,...,...,...,...,...
9183,19996783,22140408,1,C259,10,10,"Malignant neoplasm of pancreas, unspecified"
9184,19997538,22701415,1,C20,10,10,Malignant neoplasm of rectum
9185,19999204,29046609,1,C220,10,10,Liver cell carcinoma
9186,19999287,22997012,1,1629,9,9,"Malignant neoplasm of bronchus and lung, unspe..."


# patients파일

In [27]:
df_patients = pd.read_csv("patients.csv")
df_patients.head(3)

,subject_id,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10000032,F,52,2180,2014 - 2016,2180-09-09
1,10000048,F,23,2126,2008 - 2010,NaN
2,10000068,F,19,2160,2008 - 2010,NaN


# result와 patients 병합

In [28]:
merge2 = pd.merge(result,df_patients, on=['subject_id'])
merge2

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title,gender,anchor_age,anchor_year,anchor_year_group,dod
0,10001401,21544441,1,C675,10,10,Malignant neoplasm of bladder neck,F,89,2131,2014 - 2016,NaN
1,10001877,25679292,1,2252,9,9,Benign neoplasm of cerebral meninges,M,89,2149,2008 - 2010,NaN
2,10002155,20345487,1,1628,9,9,Malignant neoplasm of other parts of bronchus ...,F,80,2128,2008 - 2010,2131-03-10
3,10002348,22725460,1,C7931,10,10,Secondary malignant neoplasm of brain,F,77,2112,2017 - 2019,NaN
4,10003400,20214994,1,1543,9,9,"Malignant neoplasm of anus, unspecified site",F,72,2134,2011 - 2013,2137-09-02
...,...,...,...,...,...,...,...,...,...,...,...,...
9183,19996783,22140408,1,C259,10,10,"Malignant neoplasm of pancreas, unspecified",M,89,2188,2017 - 2019,2188-05-21
9184,19997538,22701415,1,C20,10,10,Malignant neoplasm of rectum,M,53,2168,2017 - 2019,NaN
9185,19999204,29046609,1,C220,10,10,Liver cell carcinoma,M,61,2146,2017 - 2019,2146-08-29
9186,19999287,22997012,1,1629,9,9,"Malignant neoplasm of bronchus and lung, unspe...",F,71,2191,2008 - 2010,2197-09-02


# 환자나이 계산하기 위해 merge

In [29]:
df_admission = pd.read_csv("admissions.csv")
merge2_admission = pd.merge(merge2,df_admission, on=['subject_id','hadm_id'])
merge2_admission

,subject_id,hadm_id,seq_num,icd_code,icd_version_x,icd_version_y,long_title,gender,anchor_age,anchor_year,...,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag
0,10001401,21544441,1,C675,10,10,Malignant neoplasm of bladder neck,F,89,2131,...,P872K3,PHYSICIAN REFERRAL,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,NaN,NaN,0
1,10001877,25679292,1,2252,9,9,Benign neoplasm of cerebral meninges,M,89,2149,...,P47KFL,EMERGENCY ROOM,SKILLED NURSING FACILITY,Other,ENGLISH,MARRIED,WHITE,2149-05-21 10:58:00,2149-05-21 17:15:00,0
2,10002155,20345487,1,1628,9,9,Malignant neoplasm of other parts of bronchus ...,F,80,2128,...,P80515,EMERGENCY ROOM,DIED,Other,ENGLISH,MARRIED,WHITE,2131-03-09 19:14:00,2131-03-09 21:33:00,1
3,10002348,22725460,1,C7931,10,10,Secondary malignant neoplasm of brain,F,77,2112,...,P35WT5,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicare,ENGLISH,SINGLE,WHITE,2112-11-30 15:08:00,2112-11-30 23:24:00,0
4,10003400,20214994,1,1543,9,9,"Malignant neoplasm of anus, unspecified site",F,72,2134,...,P60ZCO,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9183,19996783,22140408,1,C259,10,10,"Malignant neoplasm of pancreas, unspecified",M,89,2188,...,P878WT,PHYSICIAN REFERRAL,HOME,Other,?,MARRIED,ASIAN - CHINESE,2188-04-22 08:36:00,2188-04-22 19:49:00,0
9184,19997538,22701415,1,C20,10,10,Malignant neoplasm of rectum,M,53,2168,...,P68URZ,PHYSICIAN REFERRAL,HOME,Other,ENGLISH,MARRIED,WHITE,NaN,NaN,0
9185,19999204,29046609,1,C220,10,10,Liver cell carcinoma,M,61,2146,...,P553JT,TRANSFER FROM HOSPITAL,HOME,Medicare,ENGLISH,SINGLE,WHITE,2146-05-30 14:10:00,2146-05-30 20:09:00,0
9186,19999287,22997012,1,1629,9,9,"Malignant neoplasm of bronchus and lung, unspe...",F,71,2191,...,P44WVR,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,2197-07-26 02:51:00,2197-07-26 05:00:00,0


# 입원시점에서 환자나이 계산

In [30]:
# 출생 연도 계산
print(merge2_admission[['anchor_year','anchor_age']].head(5))
print("~~")

##adgmit_age가 이상한 경우
#  환자의 나이가 89세 이상인 경우, 데이터 보호를 위해 모든 연령이 89세로 고정되어 있다는 것입니다.
merge2_admission = merge2_admission[merge2_admission['anchor_age']<89]

# admittime 열을 datetime 형식으로 변환
merge2_admission['birth_year'] = merge2_admission['anchor_year'] - merge2_admission['anchor_age']

# 입원 연도 추출
admit_year = pd.to_datetime(merge2_admission['admittime']).dt.year

# 입원 시점의 나이 계산
merge2_admission['admit_age'] = admit_year - merge2_admission['birth_year']

#인덱스 초기화
merge2_admission.reset_index(drop=True, inplace=True)

#결과확인
print(merge2_admission[['subject_id', 'admittime', 'admit_age','birth_year','anchor_year','anchor_age']].head())

   anchor_year  anchor_age
0         2131          89
1         2149          89
2         2128          80
3         2112          77
4         2134          72
~~
   subject_id            admittime  admit_age  birth_year  anchor_year  \
0    10002155  2131-03-09 20:33:00         83        2048         2128   
1    10002348  2112-11-30 22:22:00         77        2035         2112   
2    10003400  2137-02-24 10:00:00         75        2062         2134   
3    10004235  2196-06-14 08:30:00         47        2149         2196   
4    10005024  2138-03-29 01:17:00         60        2078         2138   

   anchor_age  
0          80  
1          77  
2          72  
3          47  
4          60  


C:\Users\TS\AppData\Local\Temp\ipykernel_11096\4117411835.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge2_admission['birth_year'] = merge2_admission['anchor_year'] - merge2_admission['anchor_age']
C:\Users\TS\AppData\Local\Temp\ipykernel_11096\4117411835.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge2_admission['admit_age'] = admit_year - merge2_admission['birth_year']


# 예측했을때 써야할 피쳐들만 가지고왔는데 의논이 필요한듯

In [31]:
merge2_admission = merge2_admission[['subject_id','hadm_id','admit_age','seq_num','gender','dod','deathtime','insurance','language','marital_status','race','hospital_expire_flag','birth_year',]]
merge2_admission

,subject_id,hadm_id,admit_age,seq_num,gender,dod,deathtime,insurance,language,marital_status,race,hospital_expire_flag,birth_year
0,10002155,20345487,83,1,F,2131-03-10,2131-03-10 21:53:00,Other,ENGLISH,MARRIED,WHITE,1,2048
1,10002348,22725460,77,1,F,NaN,NaN,Medicare,ENGLISH,SINGLE,WHITE,0,2035
2,10003400,20214994,75,1,F,2137-09-02,NaN,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,0,2062
3,10004235,25970245,47,1,M,NaN,NaN,Medicaid,ENGLISH,SINGLE,BLACK/CAPE VERDEAN,0,2149
4,10005024,25023471,60,1,M,2138-04-19,2138-04-19 11:30:00,Medicaid,ENGLISH,SINGLE,UNKNOWN,1,2078
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8954,19995127,21801907,83,1,M,2138-06-12,NaN,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,0,2055
8955,19997538,22701415,53,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2115
8956,19999204,29046609,61,1,M,2146-08-29,NaN,Medicare,ENGLISH,SINGLE,WHITE,0,2085
8957,19999287,22997012,77,1,F,2197-09-02,NaN,Medicare,ENGLISH,SINGLE,BLACK/AFRICAN AMERICAN,0,2120


# death비율과 death하지 않은비율

In [32]:
merge2_admission['hospital_expire_flag'].value_counts()

0    8608
1     351
Name: hospital_expire_flag, dtype: int64

# death 종양환자 평균나이 비교

In [33]:
print(f"종양환자의 평균나이 :{ merge2_admission['admit_age'].mean() }")

C_death = merge2_admission[merge2_admission['hospital_expire_flag']==1]
C_death_mean_age = C_death['admit_age'].mean()
print(f"death한 종양환자의 평균나이 :{C_death_mean_age}")

C_not_death = merge2_admission[merge2_admission['hospital_expire_flag']== 0]
C_not_death_mean_age = C_not_death['admit_age'].mean()
print(f"death하지않은 종양환자의 평균나이 :{C_not_death_mean_age}")

종양환자의 평균나이 :66.89541243442349
death한 종양환자의 평균나이 :68.84900284900284
death하지않은 종양환자의 평균나이 :66.81575278810409


# omr(환자 몸무게 혈압 등등) merge

In [119]:
omr = pd.read_csv("omr.csv")
# omr
merge3_omr = pd.merge(merge2_admission,omr, on=['subject_id'])
merge3_omr

,subject_id,hadm_id,admit_age,seq_num_x,gender,dod,deathtime,insurance,language,marital_status,race,hospital_expire_flag,birth_year,chartdate,seq_num_y,result_name,result_value
0,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2192-11-26,1,Blood Pressure,150/80
1,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2193-01-22,1,Blood Pressure,150/80
2,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2193-02-19,1,Blood Pressure,154/92
3,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2193-02-19,1,BMI (kg/m2),24.5
4,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2120,2193-02-19,1,Height (Inches),71.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985909,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2126,2187-07-21,1,Weight (Lbs),151.8
985910,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2126,2187-08-18,1,Blood Pressure,124/56
985911,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2126,2187-08-18,1,BMI (kg/m2),26.9
985912,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,WHITE,0,2126,2187-08-18,1,Height (Inches),64


# pharmacy(환자 약물 처방 및 투약 내역) merge

In [120]:
pharmacy = pd.read_csv("pharmacy.csv")
pharmacy = pharmacy[['subject_id', 'hadm_id','poe_id','medication','frequency','doses_per_24_hrs']]
# pharmacy
merge4_pharmacy = pd.merge(merge3_omr,pharmacy, on=['subject_id','hadm_id'])
merge4_pharmacy

/tmp/ipykernel_37756/2794743079.py:1: DtypeWarning: Columns (16,18,24,26) have mixed types. Specify dtype option on import or set low_memory=False.
  pharmacy = pd.read_csv("pharmacy.csv")


,subject_id,hadm_id,admit_age,seq_num_x,gender,dod,deathtime,insurance,language,marital_status,...,hospital_expire_flag,birth_year,chartdate,seq_num_y,result_name,result_value,poe_id,medication,frequency,doses_per_24_hrs
0,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-67,Labetalol,ONCE,1.0
1,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-191,Acetaminophen,TID,3.0
2,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-215,NaN,ASDIR,NaN
3,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-111,Metoprolol Tartrate,Q4H:PRN,NaN
4,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2120,2192-11-26,1,Blood Pressure,150/80,11282936-50,HYDROmorphone (Dilaudid),Q5MIN:PRN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56003648,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2126,2187-08-18,1,Weight (Lbs),156.8,19290501-328,NaN,ASDIR,NaN
56003649,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2126,2187-08-18,1,Weight (Lbs),156.8,19290501-621,PredniSONE,DAILY,1.0
56003650,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2126,2187-08-18,1,Weight (Lbs),156.8,19290501-419,Metoprolol Tartrate,ONCE MR1,1.0
56003651,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,0,2126,2187-08-18,1,Weight (Lbs),156.8,19290501-726,Haloperidol,ONCE MR1,1.0


# prescriptions(약물 처방 기록) merge

In [126]:
prescriptions = pd.read_csv("prescriptions.csv")
prescriptions = prescriptions[['subject_id', 'hadm_id','poe_id','drug','dose_val_rx','form_unit_disp','doses_per_24_hrs']]
merge5_prescriptions = pd.merge(merge4_pharmacy,prescriptions, on=['subject_id','hadm_id','poe_id'])
merge5_prescriptions

/tmp/ipykernel_37756/1010516667.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  prescriptions = pd.read_csv("prescriptions.csv")


,subject_id,hadm_id,admit_age,seq_num_x,gender,dod,deathtime,insurance,language,marital_status,...,result_name,result_value,poe_id,medication,frequency,doses_per_24_hrs_x,drug,dose_val_rx,form_unit_disp,doses_per_24_hrs_y
0,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,Blood Pressure,150/80,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
1,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,Blood Pressure,150/80,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
2,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,Blood Pressure,154/92,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
3,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,BMI (kg/m2),24.5,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
4,11282936,22922093,73,1,M,NaN,NaN,Other,ENGLISH,MARRIED,...,Height (Inches),71.5,11282936-67,Labetalol,ONCE,1.0,Labetalol,20,VIAL,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97646596,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,Weight (Lbs),151.8,19290501-962,Acetaminophen IV,Q8H,3.0,Acetaminophen IV,1000,VIAL,3.0
97646597,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,Blood Pressure,124/56,19290501-962,Acetaminophen IV,Q8H,3.0,Acetaminophen IV,1000,VIAL,3.0
97646598,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,BMI (kg/m2),26.9,19290501-962,Acetaminophen IV,Q8H,3.0,Acetaminophen IV,1000,VIAL,3.0
97646599,19290501,28187167,58,1,F,NaN,NaN,Other,ENGLISH,MARRIED,...,Height (Inches),64,19290501-962,Acetaminophen IV,Q8H,3.0,Acetaminophen IV,1000,VIAL,3.0
